# (04) PyTorch profiler

**Motivation**: @torch.jit.script was slowing things down <br>

In [1]:
# HIDE CODE


import os, sys
from IPython.display import display

# tmp & extras dir
git_dir = os.path.join(os.environ['HOME'], 'Dropbox/git')
extras_dir = os.path.join(git_dir, 'jb-MTMST/_extras')
fig_base_dir = os.path.join(git_dir, 'jb-MTMST/figs')
tmp_dir = os.path.join(git_dir, 'jb-MTMST/tmp')

# GitHub
sys.path.insert(0, os.path.join(git_dir, '_MTMST'))
from model.train_vae import TrainerVAE, ConfigTrainVAE
from model.vae2d import VAE, ConfigVAE
from analysis.opticflow import *
from figures.fighelper import *

# warnings, tqdm, & style
warnings.filterwarnings('ignore', category=DeprecationWarning)
from tqdm.notebook import tqdm
from rich.jupyter import print
%matplotlib inline
set_style()

## Trainer

In [2]:
vae = VAE(ConfigVAE(
    n_latent_scales=2, n_groups_per_scale=20, n_latent_per_group=7,
    scale_init=False, residual_kl=True, ada_groups=True, # separable=False,
))
self = TrainerVAE(
    model=vae,
    cfg=ConfigTrainVAE(
        lr=0.002, batch_size=500, epochs=2000, grad_clip=None,
        lambda_anneal=True, lambda_init=1e-7, lambda_norm=1e-2,
        kl_beta=0.25, kl_anneal_cycles=1, 
        scheduler_kws={'T_max': 660.0, 'eta_min': 1e-05},   
        optimizer='adamax_fast',
    ),
    device='cuda:1',
)
vae.cfg.total_latents()

210

In [3]:
vae.print()
vae.scales

+--------------+------------+
| Module Name  | Num Params |
+--------------+------------+
|     VAE      |  17.0 Mil  |
|     ---      |    ---     |
|     stem     |   1.1 K    |
| pre_process  |   96.0 K   |
|  enc_tower   |  9.4 Mil   |
|     enc0     |   16.6 K   |
| enc_sampler  |  1.4 Mil   |
| dec_sampler  |  1.4 Mil   |
|    expand    |   71.0 K   |
|  dec_tower   |  4.5 Mil   |
| post_process |   40.0 K   |
|     out      |    578     |
+--------------+------------+

[8, 4]

In [4]:
len(vae.all_conv_layers), len(vae.all_log_norm)

(291, 287)

In [5]:
from tqdm import tqdm

In [ ]:
# neither epe, nor _normalize have jit. furthermore, _normalize uses linalg.vector_norm

In [6]:
%%time

self.train(100, 'test', False)

epoch # 100, avg loss: 4.861284: 100%|██████████| 100/100 [1:49:38<00:00, 65.79s/it]


CPU times: user 1h 47min 2s, sys: 3min 38s, total: 1h 50min 41s
Wall time: 1h 49min 39s


In [6]:
%%time

self.model.train()

for _ in range(5):
    for x, norm in tqdm(iter(self.dl_trn)):
        self.model(x)

100%|██████████| 80/80 [00:14<00:00,  5.67it/s]

CPU times: user 1min 21s, sys: 1.53 s, total: 1min 22s
Wall time: 1min 20s


In [7]:
# was with bunch of jit at Normal

100%|██████████| 80/80 [00:15<00:00,  5.00it/s]

CPU times: user 1min 26s, sys: 1.65 s, total: 1min 27s
Wall time: 1min 25s


In [6]:
# was with torch.linalg.vector_norm

100%|██████████| 80/80 [00:11<00:00,  6.88it/s]

CPU times: user 1min 4s, sys: 1.64 s, total: 1min 6s
Wall time: 1min 3s


In [7]:
from model.utils_model import kl_balancer

In [9]:
self.pbar = tqdm(range(10))

  0%|          | 0/10 [00:00<?, ?it/s]

In [10]:
%%time

for e in range(5): 
    self.iteration(0, n_iters_warmup=0)

gstep # 61, nelbo: 15.907, grad: 228.0:   0%|          | 0/10 [05:06<?, ?it/s]      

KeyboardInterrupt: 

In [9]:
%%time

self.model.train()

for e in range(5): 
    for i, (x, norm) in tqdm(enumerate(iter(self.dl_trn))):
        gstep = e * len(self.dl_trn) + i
        y, _, q, p = self.model(x)
        epe = self.model.loss_recon(x=x, y=y, w=1/norm)
        kl_all, kl_diag = self.model.loss_kl(q, p)
        # balance kl
        balanced_kl, gamma, kl_vals = kl_balancer(
            kl_all=kl_all,
            alpha=self.alphas,
            coeff=self.betas[gstep],
            beta=self.cfg.kl_beta,
        )
        loss = torch.mean(epe + balanced_kl)
        # add regularization
        loss_w = self.model.loss_weight()
        if self.wd_coeffs[gstep] > 0:
            loss += self.wd_coeffs[gstep] * loss_w
        cond_reg_spectral = self.cfg.lambda_norm > 0 \
            and self.cfg.spectral_reg and \
            not self.model.cfg.spectral_norm
        if cond_reg_spectral:
            loss_sr = self.model.loss_spectral(
                device=self.device, name='w')
            loss += self.wd_coeffs[gstep] * loss_sr
        else:
            loss_sr = None
        loss.backward()

80it [00:54,  1.48it/s]
80it [00:43,  1.84it/s]
80it [00:44,  1.78it/s]
80it [00:48,  1.65it/s]
80it [00:44,  1.79it/s]

CPU times: user 3min 51s, sys: 6.77 s, total: 3min 58s
Wall time: 3min 55s


In [7]:
# was with torch.linalg.vector_norm

80it [00:51,  1.54it/s]
80it [00:46,  1.72it/s]
80it [00:42,  1.88it/s]
80it [00:46,  1.73it/s]
80it [00:48,  1.63it/s]

CPU times: user 3min 50s, sys: 8.81 s, total: 3min 59s
Wall time: 3min 56s


In [9]:
activities = [
    torch.profiler.ProfilerActivity.CPU,
    torch.profiler.ProfilerActivity.CUDA,
]
kws = dict(
    activities=activities,
    record_shapes=True,
    with_stack=True,
)
with torch.profiler.profile(**kws) as prof:
    y, _, q, p = self.model(x)
    epe = self.model.loss_recon(x=x, y=y, w=1/norm)
    kl_all, kl_diag = self.model.loss_kl(q, p)
    # balance kl
    balanced_kl, gamma, kl_vals = kl_balancer(
        kl_all=kl_all,
        alpha=self.alphas,
        coeff=self.betas[gstep],
        beta=self.cfg.kl_beta,
    )
    loss = torch.mean(epe + balanced_kl)
    # add regularization
    loss_w = self.model.loss_weight()
    if self.wd_coeffs[gstep] > 0:
        loss += self.wd_coeffs[gstep] * loss_w
    cond_reg_spectral = self.cfg.lambda_norm > 0 \
        and self.cfg.spectral_reg and \
        not self.model.cfg.spectral_norm
    if cond_reg_spectral:
        loss_sr = self.model.loss_spectral(
            device=self.device, name='w')
        loss += self.wd_coeffs[gstep] * loss_sr
    else:
        loss_sr = None
    loss.backward()

In [10]:
print(prof.key_averages(group_by_stack_n=5).table(sort_by="self_cuda_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  
------------  ------------  ------------  ------------  ------------  ------------  
------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU 
total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg 
# of Calls  
-------------------------------------------------------  ------------  ------------  
------------  ------------  ------------  ------------  ------------  ------------  
------------  ------------  
                             aten::convolution_backward         5.90%      56.383ms        
11.40%     108.907ms     310.276us     208.844ms        49.17%     218.653ms     622.943us   
351  
                                aten::cudnn_convolution         5.00%      47.745ms         
7.45%      71.201ms     221.810us      71.497ms        16.83%     105.171ms     327.636us    
321  
void cudnn::detail::dgrad_alg1_engine<float, 128, 6,...         0.00%       0.000us         
0.00%       0.000us       0.000us      63.215ms        14.88%      63.215ms       1.054ms    
60  
                                   volta_sgemm_64x64_nt         0.00%       0.000us         
0.00%       0.000us       0.000us      46.713ms        11.00%      46.713ms     126.593us    
369  
                                       cudaLaunchKernel        26.61%     254.222ms        
26.61%     254.222ms      16.282us      43.567ms        10.26%      43.567ms       2.790us   
15614  
                                             aten::add_         3.54%      33.856ms         
6.90%      65.932ms      26.205us      32.449ms         7.64%      34.505ms      13.714us    
2516  
void at::native::vectorized_elementwise_kernel<4, at...         0.00%       0.000us         
0.00%       0.000us       0.000us      31.880ms         7.51%      31.880ms      12.468us    
2557  
void cudnn::winograd_nonfused::winogradForwardData4x...         0.00%       0.000us         
0.00%       0.000us       0.000us      31.435ms         7.40%      31.435ms      85.190us    
369  
void cudnn::winograd_nonfused::winogradForwardOutput...         0.00%       0.000us         
0.00%       0.000us       0.000us      26.804ms         6.31%      26.804ms      72.640us    
369  
                                              aten::mul         5.81%      55.523ms         
9.15%      87.372ms      37.725us      21.854ms         5.15%      26.911ms      11.620us    
2316  
-------------------------------------------------------  ------------  ------------  
------------  ------------  ------------  ------------  ------------  ------------  
------------  ------------  
Self CPU time total: 955.230ms
Self CUDA time total: 424.738ms

In [11]:
print(prof.key_averages(group_by_stack_n=5).table(sort_by="self_cpu_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  
------------  ------------  ------------  ------------  ------------  ------------  
------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU 
total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg 
# of Calls  
-------------------------------------------------------  ------------  ------------  
------------  ------------  ------------  ------------  ------------  ------------  
------------  ------------  
                                       cudaLaunchKernel        26.61%     254.222ms        
26.61%     254.222ms      16.282us      43.567ms        10.26%      43.567ms       2.790us   
15614  
                             aten::convolution_backward         5.90%      56.383ms        
11.40%     108.907ms     310.276us     208.844ms        49.17%     218.653ms     622.943us   
351  
                                              aten::mul         5.81%      55.523ms         
9.15%      87.372ms      37.725us      21.854ms         5.15%      26.911ms      11.620us    
2316  
                                aten::cudnn_convolution         5.00%      47.745ms         
7.45%      71.201ms     221.810us      71.497ms        16.83%     105.171ms     327.636us    
321  
                                              aten::div         4.57%      43.701ms         
6.90%      65.879ms      39.331us       6.326ms         1.49%      10.167ms       6.070us    
1675  
                                             aten::add_         3.54%      33.856ms         
6.90%      65.932ms      26.205us      32.449ms         7.64%      34.505ms      13.714us    
2516  
                                              aten::sum         3.28%      31.303ms         
4.86%      46.424ms      36.962us      17.573ms         4.14%      17.573ms      13.991us    
1256  
                                            aten::addmm         2.97%      28.349ms         
4.38%      41.835ms     166.012us       1.517ms         0.36%       4.990ms      19.802us    
252  
                                              aten::add         2.82%      26.940ms         
4.29%      40.992ms      51.627us      11.840ms         2.79%      14.414ms      18.154us    
794  
                                              aten::exp         2.09%      19.971ms         
5.51%      52.658ms     180.336us     682.000us         0.16%       6.437ms      22.045us    
292  
-------------------------------------------------------  ------------  ------------  
------------  ------------  ------------  ------------  ------------  ------------  
------------  ------------  
Self CPU time total: 955.230ms
Self CUDA time total: 424.738ms

### Save to txt?

In [12]:
output = prof.key_averages(group_by_stack_n=5).table(sort_by="self_cuda_time_total", row_limit=100)
with open("profile_cuda.txt", "w") as f:
    f.write(output)
    
output = prof.key_averages(group_by_stack_n=5).table(sort_by="self_cpu_time_total", row_limit=100)
with open("profile_cpu.txt", "w") as f:
    f.write(output)

**Conclusion**: jit was slowing things down

In [ ]:
activities = [
    torch.profiler.ProfilerActivity.CPU,
    torch.profiler.ProfilerActivity.CUDA,
]
kws = dict(
    activities=activities,
    record_shapes=True,
    with_stack=True,
)
with torch.profiler.profile(**kws) as prof:
    self.train(1, save=False)

epoch # 1, avg loss: 57.204149: 100%|██████████| 1/1 [02:20<00:00, 140.16s/it]


In [ ]:
output = prof.key_averages(group_by_stack_n=5).table(sort_by="self_cuda_time_total", row_limit=100)
with open("profile_cuda.txt", "w") as f:
    f.write(output)
    
output = prof.key_averages(group_by_stack_n=5).table(sort_by="self_cpu_time_total", row_limit=100)
with open("profile_cpu.txt", "w") as f:
    f.write(output)

## Compile

In [3]:
vae = VAE(ConfigVAE(
    n_latent_scales=2, n_groups_per_scale=20, n_latent_per_group=7,
    scale_init=False, residual_kl=True, ada_groups=True, # separable=False,
))
tr = TrainerVAE(
    model=torch.compile(vae),
    cfg=ConfigTrainVAE(
        lr=0.002, batch_size=500, epochs=2000, grad_clip=1000,
        lambda_anneal=True, lambda_init=1e-7, lambda_norm=1e-2,
        kl_beta=0.25, kl_anneal_cycles=1, 
        scheduler_kws={'T_max': 660.0, 'eta_min': 1e-05},   
        optimizer='adamax_fast',
    ),
    device='cuda:1',
)
vae.cfg.total_latents()

210

In [4]:
%%time

tr.model.train()

for _ in range(5):
    for x, norm in tqdm(iter(tr.dl_trn)):
        tr.model(x)

  0%|          | 0/80 [00:00<?, ?it/s]

BackendCompilerFailed: debug_wrapper raised RuntimeError: Function ConvolutionBackward0 returned an invalid gradient at index 1 - expected device cuda:1 but got cuda:0

Set torch._dynamo.config.verbose=True for more information


You can suppress this exception and fall back to eager by setting:
    torch._dynamo.config.suppress_errors = True


In [ ]:
tr.train(comment='compiled_test')